In [1]:
import os
from collections import defaultdict

import helper
import numpy as np
import torch
from torchvision.models import resnet50
from tqdm import tqdm

torch.manual_seed(0)
device = "cuda" if torch.cuda.is_available() else "cpu"
WEIGHT_PATH = "weights"
RECORD_PATH = "records"
os.makedirs(WEIGHT_PATH, exist_ok=True)
os.makedirs(RECORD_PATH, exist_ok=True)

In [2]:
trainset = helper.SimCLRDataset("birds-400/train")
validset = helper.SimCLRDataset("birds-400/valid")
testset = helper.SimCLRDataset("birds-400/test")

model = resnet50()
model.fc = torch.nn.Sequential(
    torch.nn.Linear(2048, 512), torch.nn.ReLU(), torch.nn.Linear(512, 128)
)

model = model.to(device)

trainloader = torch.utils.data.DataLoader(trainset, 32, True, pin_memory=True)
validloader = torch.utils.data.DataLoader(validset, 32, pin_memory=True)
testloader = torch.utils.data.DataLoader(testset, 32, pin_memory=True)


optimizer = torch.optim.SGD(model.parameters(), 1e-3, momentum=0.9, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, len(trainloader) * 50)
criterion = helper.NTXentLoss()

In [3]:
history = []
for i in range(1, 101):
    print(f"Epoch {i}/100")
    metric = defaultdict(list)
    pbar = tqdm(total=len(trainloader))

    model.train()
    for i, (inputs_1, inputs_2) in enumerate(trainloader, 1):
        inputs_1 = inputs_1.to(device)
        inputs_2 = inputs_2.to(device)

        outputs_1 = model(inputs_1)
        outputs_2 = model(inputs_2)
        loss = criterion(outputs_1, outputs_2)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        metric["train_loss"].append(loss.item())
        pbar.set_postfix({k: np.mean(v) for k, v in metric.items()})
        pbar.update()

    model.eval()
    for i, (inputs_1, inputs_2) in enumerate(validloader, 1):
        inputs_1 = inputs_1.to(device)
        inputs_2 = inputs_2.to(device)

        with torch.inference_mode():
            outputs_1 = model(inputs_1)
            outputs_2 = model(inputs_2)
            loss = criterion(outputs_1, outputs_2)

        metric["valid_loss"].append(loss.item())
        pbar.set_postfix({k: np.mean(v) for k, v in metric.items()})

    history.append({k: np.mean(v) for k, v in metric.items()})
    pbar.close()
torch.save(model.state_dict(), os.path.join(WEIGHT_PATH, "r50_simclr_pretrain.pt"))
torch.save(history, os.path.join(RECORD_PATH, "r50_simclr_pretrain.pt"))

Epoch 1/100


100%|██████████| 1825/1825 [16:24<00:00,  1.85it/s, train_loss=2.55, valid_loss=2.12]


Epoch 2/100


100%|██████████| 1825/1825 [16:27<00:00,  1.85it/s, train_loss=1.69, valid_loss=1.87]


Epoch 3/100


...

100%|██████████| 1825/1825 [16:27<00:00,  1.85it/s, train_loss=0.369, valid_loss=0.713]


Epoch 99/100


100%|██████████| 1825/1825 [16:25<00:00,  1.85it/s, train_loss=0.356, valid_loss=0.883]


Epoch 100/100


100%|██████████| 1825/1825 [16:23<00:00,  1.86it/s, train_loss=0.353, valid_loss=0.728]
